#### The goal of this project file is to predict nitrogen, phosphorus, and sediment concentrations at subwatersheds 8, 11, and 12

- so far only completed nitrogen

In [1]:
import pandas as pd
import math
import numpy as np
import sklearn as sk
import sklearn.datasets as skd
import sklearn.ensemble as ske
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
%matplotlib inline

#### Main nutrients dataset

In [2]:
df_main = pd.read_excel('2016-17 raw data.xlsx', sheet_name = 'By subwatershed')
#df_main.head()

In [3]:
df_main = df_main.rename(columns={'Row crop area (%)':'Row crop','Drainage extend (%)':'drain extend',
                                  'Soils with C-D drain classes area (%)':'C-D soils',
                                  'Manure application area (%)':'Manure', 'Nutrient mgmt area (%)':'Nutrient mgmt',
                                  'Cover crop area (%)':'Cover crop','Reduced till area (%)':'Reduced till',
                                  'Terraces area (%)':'Terraces', 'GWW area (%)':'GWW', 'CRP area (%)':'CRP',
                                  'Flow (cms)':'Flow', 'NH3-N (mg N/L)':'NH3','DRP (mg P/L)':'DRP', 'TP (mg P/L)':'TP', 
                                  'NOx-N (mg N/L)':'NOx', 'TN (mg N/L)':'TN', 'TSS (mg/L)':'TSS', 'VSS (mg/L)':'VSS'})

df_main.head()

,Site,Row crop,drain extend,C-D soils,Manure,Nutrient mgmt,Cover crop,Reduced till,Terraces,GWW,...,Sample date,Sample type,Flow,DRP,TP,TSS,VSS,NH3,NOx,TN
0,Sub11,0.93,0.234,0.251,0.543494,0.27256,0.0,0.037315,0.000823,0.044298,...,2016-03-31,Base,0.025511,0.001,0.293,18.666667,14.000000,0.022,35.3001,34.2560
1,Sub11,0.93,0.234,0.251,0.543494,0.27256,0.0,0.037315,0.000823,0.044298,...,2016-04-14,Base,0.041794,0.013,0.040,2.000000,3.333333,0.002,31.7656,31.7656
2,Sub11,0.93,0.234,0.251,0.543494,0.27256,0.0,0.037315,0.000823,0.044298,...,2016-04-28,Base,0.078278,0.001,0.129,39.333333,31.333333,0.002,30.5188,30.3610
3,Sub11,0.93,0.234,0.251,0.543494,0.27256,0.0,0.037315,0.000823,0.044298,...,2016-05-17,Base,0.064980,0.001,0.177,63.333333,37.333333,0.002,22.9285,23.2680
4,Sub11,0.93,0.234,0.251,0.543494,0.27256,0.0,0.037315,0.000823,0.044298,...,2016-05-31,Base,0.052347,0.001,0.017,1.666667,1.000000,0.019,15.6345,14.2820


In [4]:
df_main.describe()

,Row crop,drain extend,C-D soils,Manure,Nutrient mgmt,Cover crop,Reduced till,Terraces,GWW,CRP,Flow,DRP,TP,TSS,VSS,NH3,NOx,TN
count,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000
mean,0.842857,0.248265,0.290673,0.238215,0.394014,0.027649,0.023257,0.001262,0.016176,0.036380,0.070581,0.032425,0.147747,42.881374,35.011819,0.078260,17.698664,19.615911
std,0.081114,0.009093,0.029888,0.218628,0.325860,0.038064,0.017787,0.000367,0.018357,0.027366,0.090905,0.073063,0.239218,112.207070,97.326136,0.411982,8.409922,10.120425
min,0.740000,0.234000,0.251000,0.000000,0.058427,0.000000,0.000000,0.000823,0.000000,0.008816,0.000600,0.001000,0.001500,0.500000,0.500000,0.002000,0.000000,0.900000
25%,0.740000,0.234000,0.251000,0.000000,0.058427,0.000000,0.000000,0.000823,0.000000,0.008816,0.018530,0.001000,0.027000,5.333333,3.316667,0.002000,9.650000,9.800000
50%,0.880000,0.253000,0.287000,0.239038,0.272560,0.000000,0.036303,0.001148,0.010146,0.025144,0.044409,0.005000,0.055000,11.666667,8.300000,0.016200,18.839000,19.650000
75%,0.930000,0.255000,0.325000,0.543494,0.805423,0.079694,0.037315,0.001710,0.044298,0.071151,0.090678,0.026000,0.157000,29.016667,24.000000,0.043800,24.847850,26.540368
max,0.930000,0.255000,0.325000,0.543494,0.805423,0.079694,0.037315,0.001710,0.044298,0.071151,0.757435,0.436000,1.568000,809.000000,720.000000,4.898300,35.300100,48.600000


#### Auxiliary dataset
##### Factors that can be potentially used for predicting N concentration
- Daily precipitation
- Antecedent precipitation
- Seasonality
- Flow (likely low impact on NOx and TN)
- Temperature (likely low impact on NOx and TN)

In [5]:
df_weather = pd.read_excel('Weather data.xlsx', sheet_name = 'Weather')
#df_weather.head()

### Model 1 - Impact of land managements/BMPs on nitrogen concentration

In [ ]:
# subsectioning nitrogen data
df_nitrogen = df_main.drop(columns=['DRP','TP', 'TSS', 'VSS'])

# extract year and month from sample dates, then visualize the N concentration distribution by sites and years
df_nitrogen['Year'] = pd.DatetimeIndex(df_nitrogen['Sample date']).year
df_nitrogen['Month'] = pd.DatetimeIndex(df_nitrogen['Sample date']).month
df_nitrogen['Day'] = df_main['Sample date'].dt.dayofyear
#df_nitrogen.head()

In [ ]:
# visualizing N concentration distribution by sites
#list_of_analytes = ['NH3', 'NOx', 'TN']

#for x in list_of_analytes:
    #ax = sns.boxplot(x='Site', y=x, data=df_nitrogen)
    #plt.show()

In [ ]:
#list_of_analytes = ['NH3', 'NOx', 'TN']

#for x in list_of_analytes:
    #ax = sns.boxplot(x='Year', y=x, data=df_nitrogen, hue='Site')
    #plt.show()

In [ ]:
# convert Sample date from timestamp to numeric because sklearn cannot process timestamp format
df_nitrogen['Sample date'] = pd.to_numeric(pd.to_datetime(df_nitrogen['Sample date']))

# implement random forest
from sklearn.model_selection import train_test_split

# Use watershed characteristics to predict TN concentration
# other factors that can be used: seasonality, day of year, month of year, etc. 
# tried using day and month of the year as predictors - both are bad predictors
# next step: include weather data (require to be pre-processed, such as calculating daily precip, antecedent precip, etc. - need to have same length as df_nitrogen dataset)
predictors = ['Row crop', 'drain extend','C-D soils','Manure','Nutrient mgmt','Cover crop','Reduced till','Terraces',
                 'GWW','CRP', 'Sample date', 'Sample type','Flow']
X = df_nitrogen[predictors]

# setting TN concentration as the "target"
y = df_nitrogen['TN']

X_train, X_test, Y_train, Y_test = train_test_split (X, y, test_size = 0.25, random_state = 0)

reg = ske.RandomForestRegressor(n_estimators = 1000, random_state = 0)

# transform string data (i.e., site and sample type) into incremental value using label encoder
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
for i in range(len(X_train)):
    X_train[['Sample type']]= X_train[['Sample type']].apply(le.fit_transform)
    X_test[['Sample type']]= X_test[['Sample type']].apply(le.fit_transform)

reg.fit(X_train, Y_train)

Y_pred = reg.predict(X_test)

from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

print('training period explained variance:', explained_variance_score(Y_test, Y_pred))
print('training period mean abs error:',mean_absolute_error(Y_test, Y_pred))
print('training period mean squared error:',mean_squared_error(Y_test, Y_pred))
print('training period r2:',r2_score(Y_test, Y_pred))

fet_ind = np.argsort(reg.feature_importances_)[::-1]
fet_imp = reg.feature_importances_[np.argsort(reg.feature_importances_)][::-1]

fig, ax = plt.subplots(1, 1, figsize=(8,3))
labels = np.asarray(X_train.columns[fet_ind])
pd.Series(fet_imp, index = labels).plot('bar', ax=ax)
ax.set_title('Feature importance')

In [ ]:
#fig, ax = plt.subplots(1, 1, figsize=(8,3))
#ax.scatter(X['CRP'],y)
#ax.set_xlabel('Flow')
#ax.set_ylabel('NOx conc')

## Try to predict 2018 data, then compare against actual data

In [ ]:
df_testraw = pd.read_excel('Test data 2018.xlsx', sheet_name = 'By subwatershed')
df_test = df_testraw.drop(columns=['Site'])

# transform string data (i.e., site and sample type) into incremental value using label encoder
le = preprocessing.LabelEncoder()
for i in range(len(df_test['Sample type'])):
    df_test[['Site']]= df_test[['Site']].apply(le.fit_transform)
    df_test[['Sample type']]= df_test[['Sample type']].apply(le.fit_transform)

# convert Sample date from timestamp to numeric because sklearn cannot process timestamp format
df_test['Sample date'] = pd.to_numeric(pd.to_datetime(df_test['Sample date']))    
    
# tried adding month as predictor - bad predictor
#df_test['Month'] = pd.DatetimeIndex(df_test['Sample date']).month

# convert Sample date to day of the year
# tried adding day of the year as predictor - bad predictor
# df_test['Day'] = df_test['Sample date'].dt.dayofyear





df_test.head()

In [ ]:
Y_pred = reg.predict(df_test)

df_predict = pd.DataFrame(Y_pred, columns=['TN_conc_pred'])
df_predict = pd.merge(df_testraw, df_predict, left_index=True, right_index=True)

df_actual = pd.read_excel('Test data 2018.xlsx', sheet_name = 'By subwatershed_full')
df_actual = df_actual[['Site','Sample date','Sample type', 'TN (mg N/L)']]
df_actual['day of year'] = df_actual['Sample date'].dt.dayofyear

df_predict = pd.merge(df_actual, df_predict['TN_conc_pred'], left_index=True, right_index=True)

print('prediction period explained variance:',explained_variance_score(df_predict['TN (mg N/L)'], df_predict['TN_conc_pred']))
print('prediction period mean abs error:',mean_absolute_error(df_predict['TN (mg N/L)'], df_predict['TN_conc_pred']))
print('prediction period mean squared error:',mean_squared_error(df_predict['TN (mg N/L)'], df_predict['TN_conc_pred']))
print('prediction period r2:',r2_score(df_predict['TN (mg N/L)'], df_predict['TN_conc_pred']))

plt.scatter(df_predict['TN (mg N/L)'], df_predict['TN_conc_pred'])
plt.xlabel('Actual TN conc')
plt.ylabel('Predicted TN conc')

In [ ]:
writer = pd.ExcelWriter('predict_output 2018.xlsx')
# write dataframe to excel
df_predict.to_excel(writer)
# save the excel
writer.save()

## Create a dummy dataset consisting of different levels of BMPs, then use the ML model to predict nitrogen concentrations.

In [ ]:
df_testraw = pd.read_excel('Test data dummy.xlsx', sheet_name = 'Input')
df_test = df_testraw.drop(columns=['Site'])

# convert Sample date to day of the year
df_test['Day'] = df_test['Sample date'].dt.dayofyear
df_test = df_test.drop(columns=['Sample date'])

# convert Sample date from timestamp to numeric because sklearn cannot process timestamp format
#df_test['Sample date'] = pd.to_numeric(pd.to_datetime(df_test['Sample date']))

# transform string data (i.e., site and sample type) into incremental value using label encoder
le = preprocessing.LabelEncoder()
for i in range(len(df_test['Sample type'])):
    #df_test[['Site']]= df_test[['Site']].apply(le.fit_transform)
    df_test[['Sample type']]= df_test[['Sample type']].apply(le.fit_transform)
#df_test.head()

In [ ]:
Y_pred = reg.predict(df_test)

from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

fet_ind = np.argsort(reg.feature_importances_)[::-1]
fet_imp = reg.feature_importances_[np.argsort(reg.feature_importances_)][::-1]

fig, ax = plt.subplots(1, 1, figsize=(8,3))
labels = np.asarray(X_train.columns[fet_ind])
pd.Series(fet_imp, index = labels).plot('bar', ax=ax)
ax.set_title('Feature importance')

In [ ]:
df_predict = pd.DataFrame(Y_pred, columns=['TN_conc_pred'])
df_predict = pd.merge(df_testraw, df_predict, left_index=True, right_index=True)

writer = pd.ExcelWriter('predict_output.xlsx')
# write dataframe to excel
df_predict.to_excel(writer)
# save the excel
writer.save()

### Model 2 - mpact of precipitation on flow and nitrogen concentration

In [ ]:
#df_weather.head()

In [ ]:
#df_weather['Year'], df_weather['Month'],df_weather['Day'] = df_weather['Date'].dt.year, df_weather['Date'].dt.month, df_weather['Date'].dt.day
#df_weather.head()